# Key word extraction algorithm for claims

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os, json
from helper_function import count_valid_posts, get_claims, get_premises
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

In [3]:
data_file_path = '../data/v4/Should-I-invest-in-Bitcoin_with_labels_v4_70_filled.json'
data_file_path = '../data/v4/Would-you-get-into-a-self-driving-car_v4_70_filled.json'
# data_file_path = "../data/v5/bitcoin-invest-aligned.json"
data_file_path = "../data/v5/auto-driving-aligned.json"

with open(data_file_path, 'r') as f:
    file_content = json.load(f)
print('Get valid posts: ', count_valid_posts(file_content))
claims = get_claims(file_content)
premises = get_premises(file_content)


Get valid posts:  (66, 66)


## Keyword extraction algorithms

In [4]:
# from keyword_extractor import TfIdfExtractor

# if_idf = TfIdfExtractor()

In [5]:
# if_idf.extract_keywords(premises)

## KeyBERT Extractor

In [6]:
from keyword_extractor import KeyBERTExtractor

In [7]:
keybert_extractor = KeyBERTExtractor()

In [8]:
# keybert_extractor.extract_keywords(premises)

## Extract Keywords under a certain center claim

In [9]:
# first build a list of supporting premises
claims_with_support = {}
for l in get_premises(file_content, return_list_of_strings=False):
    # l is a list of dict objects
    for o in l:
        clm, prm = o['supportClaim'], o['content']
        if not clm in claims_with_support:
            claims_with_support[clm] = []
        claims_with_support[clm].append(prm)

claim_center_relation = {}
for l in get_claims(file_content, return_list_of_strings=False):
    # l is a list of dict objects
    for o in l:
        clm, sc = o['content'], o['claimCenter']
        if not sc in claim_center_relation:
            claim_center_relation[sc] = []
        claim_center_relation[sc].append(clm)

claimCenter_premise = {}
for cs, c_list in claim_center_relation.items():
    if not cs in claimCenter_premise:
        claimCenter_premise[cs] = []
    for c in c_list:
        if c in claims_with_support:
            claimCenter_premise[cs].extend(claims_with_support[c])


In [10]:
for k,v in claimCenter_premise.items():
    print(k, ": ", len(v))

Yes, you would. :  9
the research shows people are amazingly trusting of them :  8
Not at the time of answering. :  11
Absolutely. I hate driving, and can’t wait to never have to do it again. :  16
Yes, I really would like to have a self-driving car :  37
Yes, I will. :  6
Yes, definitely. :  14
Yes, as soon as they work and are available at a reasonable price. :  5
I think that this technology has the potential to be very safe and convenient :  1
Yes :  46
What I mean is, autonomous cars would be great, IF THEY WERE THE ONLY CARS OUT THERE :  7
Absolutely not! :  11


In [11]:
output_data = []
for k,v in claimCenter_premise.items():
    term = {
        'content' : k,
        'keywords' : []
    }
    kws = keybert_extractor.extract_keywords(' '.join(v), keep_term=10)
    kws.sort(key=lambda a: a[1], reverse=True)
    term['keywords'] = kws
    output_data.append(term)
    # break

In [12]:
# claimCenter_premise['It is almost certainly in a bubble.']


In [13]:
# x = keybert_extractor.extract_keywords(' '.join(v), keep_term=15)
# x.sort(key=lambda a : a[1])
# x


In [14]:
# output_data_path = '../data/newest_data(rolling update)/claim center and statistics/bitcoin investment/keywords.json'
output_data_path = '../data/newest_data(rolling update)/claim center and statistics/automos driving/keywords.json'
with open(output_data_path, 'w') as f:
    json.dump(output_data, fp=f, indent=4)


In [15]:
# keybert_extractor.extract_keywords(" ".join(claimCenter_premise['That’s up to you.']), keep_term=10)


## Reformat

In [16]:
# sentence_sentiment = {
#     'neutral' : [
#         "It’s not too late to invest.",
#         "That’s up to you.",
#         "It depends what your level of disposable income is, how great your assets are, and what other assets you have invested in.",
#         "The significant thing is to do your own research and comprehend the dangers.",
#         "Invest in Bitcoin, only if you are okay to loss all.",
#         "Investing in Bitcoin is viable option especially in a view of current decline of the power of Fiat currencies.",
#         "If you are willing to take the risk, first make sure you understand what you are investing in and have a crypto investment strategy",
#     ],
#     "positive" : [
#         "I would say YES!",
#         "Of course you should",
#     ],
#     "negative" : [
#         "Bitcoin is pretty useless. But so is gold.",
#         "Cryto currency is an extremely high-hazard venture, and CFDs bought on margin are significantly more hazardous.",
#         "It is almost certainly in a bubble.",
#     ]
# }

# with open('../data/newest_data(rolling update)/claim center and statistics/bitcoin investment/stance_count.json', 'w') as f:
#     json.dump(sentence_sentiment, f, indent=4)

sentence_sentiment_path = '../data/newest_data(rolling update)/claim center and statistics/bitcoin investment/stance_count.json'
sentence_sentiment_path = '../data/newest_data(rolling update)/claim center and statistics/automos driving/stance_count.json'
with open(sentence_sentiment_path, 'r') as f:
    sentence_sentiment = json.load(f)

for i in output_data:
    key = i['content']
    if not key in sentence_sentiment['neutral'] and not key in sentence_sentiment['positive'] and not key in sentence_sentiment['negative']:
        print(key)


In [17]:
formated_output = {
    'neutral' : {},
    'positive' : {},
    'negative' : {}
}

for key, value in sentence_sentiment.items():
    for index, cs in enumerate(value):
        o_index = next((i for i, s in enumerate(output_data) if s['content'] == cs), -1)
        formated_output[key][f'claim_{index}'] = output_data[o_index]['keywords']

In [18]:
formated_output

{'neutral': {'claim_0': [('driving', 0.4391),
   ('trust', 0.2743),
   ('safer', 0.2704),
   ('computers', 0.2507),
   ('insurance', 0.2266),
   ('mother', 0.2101),
   ('personally', 0.1871),
   ('stanford', 0.1705),
   ('straightway', 0.1662),
   ('airport', 0.1352)],
  'claim_1': [('airplanes', 0.3614),
   ('crashes', 0.3179),
   ('driving', 0.2688),
   ('statistics', 0.2517),
   ('safety', 0.2188),
   ('mobile', 0.1629),
   ('productivity', 0.1619),
   ('plenty', 0.153),
   ('expensive', 0.1487),
   ('smarter', 0.0996)],
  'claim_2': [('driving', 0.4818),
   ('uber', 0.4494),
   ('control', 0.2702),
   ('avoid', 0.2344),
   ('curb', 0.2177),
   ('foolish', 0.1742),
   ('tedium', 0.1445),
   ('much', 0.1329),
   ('centrifugal', 0.0869),
   ('button', 0.0773)],
  'claim_3': [('driving', 0.517),
   ('parking', 0.382),
   ('insurance', 0.3205),
   ('dui', 0.3054),
   ('license', 0.3017),
   ('rights', 0.2144),
   ('unmanned', 0.1285),
   ('benders', 0.1049),
   ('watch', 0.0817),
   ('n

In [19]:
support_statistic = {}
for stm, clm in sentence_sentiment.items():
    if not stm in support_statistic:
        support_statistic[stm] = 0
    for c in clm:
        support_statistic[stm] += len(claimCenter_premise[c])
support_statistic

{'neutral': 122, 'positive': 22, 'negative': 27}

In [20]:
from datetime import date
today = date.today()
d1 = today.strftime(f'%m%d')
d1

'0830'

In [21]:
formated_output_path = f'../data/newest_data(rolling update)/claim center and statistics/bitcoin investment/keywords_formatted_{d1}_1.json'
formated_output_path = f'../data/newest_data(rolling update)/claim center and statistics/automos driving/keywords_formatted_{d1}_1.json'
with open(formated_output_path, 'w') as f:
    json.dump(formated_output, fp=f, indent=4)
